# Classifying the translated z latent embeddings
Due to the architecture of the CI-DI-VAE, we can expect a simple classifier to classify the latent embeddings. 

In [1]:
import scripts.init_gpu as init_gpu
import scripts.init_dataset as init_dataset
from scripts.triplet_functions import n_neurons
import pandas as pd


init_gpu.initialize_gpus()

locations = ['LOC2', 'LOC3']

print("Loading Dataset...")
# load the dataset
df = pd.read_csv(
    f"../dataset/processed/{locations[0]}-{locations[1]}-scaled-balanced.csv")

length = len(df.columns) - 2  # subtract the two label columns

# get train-test set
train_df, test_df, train_web_samples, test_web_samples = init_dataset.get_sample(
    df, locations, range(1500), 1200)

train_df.head()

2025-01-23 00:51:33.043643: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-23 00:51:33.059221: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737573693.067754  230991 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737573693.071439  230991 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-23 00:51:33.087029: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Num GPUs Available:  1
Loading Dataset...
Training Websites: [1309, 228, 51, 563, 501, 457, 285, 209, 1385, 1116, 178, 1209, 864, 65, 61, 191, 447, 476, 1034, 1232, 54, 1149, 407, 1466, 1330, 1436, 1490, 859, 451, 919, 1206, 569, 13, 326, 1429, 865, 696, 1468, 318, 440, 689, 1492, 189, 778, 198, 735, 704, 1236, 541, 88, 940, 1098, 255, 775, 161, 1130, 600, 1287, 1266, 740, 1182, 393, 142, 93, 1354, 466, 592, 163, 1482, 206, 1456, 1462, 928, 1301, 747, 333, 758, 727, 429, 1372, 546, 1399, 1327, 146, 1247, 1300, 350, 1093, 1495, 334, 946, 777, 552, 1310, 1140, 449, 1402, 664, 114, 469, 1486, 646, 821, 548, 135, 432, 1161, 644, 435, 1342, 1022, 810, 1316, 939, 292, 542, 1493, 505, 1478, 1103, 538, 1197, 877, 1195, 817, 741, 1404, 283, 1043, 1010, 186, 96, 224, 313, 1285, 327, 1487, 1221, 130, 788, 781, 1220, 958, 1083, 514, 1133, 23, 234, 1099, 1419, 1312, 1463, 1498, 601, 890, 323, 929, 6, 539, 1025, 365, 1039, 217, 1280, 611, 1308, 1338, 1415, 1477, 1366, 765, 330, 1104, 1086, 1, 1226, 

/home/e19372/DoH-Synthesis/code/scripts/init_dataset.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.sort_values(by=["Location"], inplace=True)


,Location,Website,0,1,2,3,4,5,6,7,...,118,119,120,121,122,123,124,125,126,127
0,LOC2,0,-0.714089,0.541197,0.391921,-0.400778,-0.266345,-0.522526,0.023889,-0.261817,...,0.125986,0.10277,0.092495,0.043004,0.158904,0.119506,0.099445,0.083843,0.037226,0.152216
1,LOC2,1006,0.093792,-1.273349,-2.142801,1.205098,1.879002,2.139593,1.482513,2.382939,...,0.125986,0.10277,0.092495,0.043004,0.158904,0.119506,0.099445,0.083843,0.037226,0.152216
2,LOC2,1006,1.401091,-1.636258,0.527106,0.354928,-0.017609,-0.049260,-0.177300,-0.412692,...,0.125986,0.10277,0.092495,0.043004,0.158904,0.119506,0.099445,0.083843,0.037226,0.152216
3,LOC2,1006,-0.831599,0.662167,0.527106,0.354928,-0.017609,-0.049260,-0.177300,-0.412692,...,0.125986,0.10277,0.092495,0.043004,0.158904,0.119506,0.099445,0.083843,0.037226,0.152216
4,LOC2,1006,0.093792,-1.152379,-2.007616,0.827245,1.630266,1.666328,1.281324,2.311938,...,0.125986,0.10277,0.092495,0.043004,0.158904,0.119506,0.099445,0.083843,0.037226,0.152216


In [6]:
 # data preprocessing for source, real target, and synthetic data
source_location, target_location = locations

target_df = test_df[test_df['Location'] == target_location]
target_df.sort_values(by=['Website'], inplace=True)
target_df.reset_index(drop=True, inplace=True)
target_df.head(20)

source_df = test_df[test_df['Location'] == source_location]
source_df.sort_values(by=['Website'], inplace=True)
source_df.reset_index(drop=True, inplace=True)


translated_z_df = pd.read_csv(f'../synthesized/{target_location}-VAE-Sampling-Z-Translations.csv')
translated_z_df = translated_z_df[translated_z_df['Location'] == target_location]
translated_z_df.sort_values(by=['Website'], inplace=True)
translated_z_df.reset_index(drop=True, inplace=True)

/tmp/ipykernel_230991/955691792.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df.sort_values(by=['Website'], inplace=True)
/tmp/ipykernel_230991/955691792.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source_df.sort_values(by=['Website'], inplace=True)


In [ ]:
# models
import tensorflow as tf
from scripts.translate_and_synthsize import SWDLoss
from scripts.train_vae import Sampling, ConvVAE_BatchNorm
latent_dim = 96
swd_loss = SWDLoss(latent_dim)
vae_model = tf.keras.models.load_model(f"../models-{locations[0]}-{locations[1]}/vae/ci_vae/ConvBased/domain_and_class/{locations[0]}-{locations[1]}-e800-mse1-kl0.01-cl1.0-ConvBatchNorm-ldim96-hdim128.keras", custom_objects={'ConvVAE_BatchNorm': ConvVAE_BatchNorm, 'Sampling': Sampling})
translator = tf.keras.models.load_model(f"../models-{locations[0]}-{locations[1]}/latent_translator/translator-kl0.01-e200.keras", custom_objects={'SWDLoss': swd_loss})

/home/e19372/anaconda3/envs/doh_synth_env/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
I0000 00:00:1737580018.961208  230991 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1737580018.961327  230991 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21349 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


In [9]:
from scripts.translate_and_synthsize import get_z_embeddings, get_z_translations

source_z = get_z_embeddings(source_df.iloc[:, 2:], vae_model)
target_z = get_z_embeddings(target_df.iloc[:, 2:], vae_model)

# Simple NN Classifier

In [8]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

def create_simple_dense_classifier(input_dim, num_classes, hidden_units=64, dropout_rate=0.1):
    """
    Creates a simple dense neural network classifier.

    Parameters:
    - input_dim: int, the number of features in the input data.
    - num_classes: int, the number of output classes.
    - hidden_units: int, the number of neurons in the hidden layer.
    - dropout_rate: float, the dropout rate for regularization.

    Returns:
    - model: A compiled Keras model.
    """
    model = Sequential([
        Dense(hidden_units, activation='relu', input_shape=(input_dim,)),
        Dropout(dropout_rate),
        Dense(hidden_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')  # Softmax for multi-class classification
    ])

    # Compile the model
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',  # Suitable for integer-encoded labels
        metrics=['accuracy']
    )

    return model


## Trained on source location z embeddings

In [26]:
from sklearn.preprocessing import LabelEncoder
import scripts.classification as classification 
le = LabelEncoder()

X_train = source_z
y_train = le.fit_transform(source_df.Website)
X_test = target_z
y_test = le.transform(target_df.Website)

model = create_simple_dense_classifier(latent_dim, len(test_web_samples))
model.fit(X_train, y_train, batch_size = 32, epochs=20, shuffle=True)

Epoch 1/100


/home/e19372/anaconda3/envs/doh_synth_env/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


929/929 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0587 - loss: 5.1893
Epoch 2/100
929/929 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4016 - loss: 2.3801
Epoch 3/100
929/929 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5433 - loss: 1.7005
Epoch 4/100
929/929 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5995 - loss: 1.4556
Epoch 5/100
929/929 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6390 - loss: 1.2958
Epoch 6/100
929/929 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6549 - loss: 1.2085
Epoch 7/100
929/929 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6727 - loss: 1.1434
Epoch 8/100
929/929 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6815 - loss: 1.1020
Epoch 9/100
929/929 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6890 - loss: 1.0897
Epoch 10/100
929/929 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6970 - loss: 1.0556
Epoch 11/100
929/929 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7043 - loss: 1.0202
Epoch 12/100
929/929 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

In [27]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Convert logits to class labels
y_pred_logits = model.predict(X_test)
y_pred = np.argmax(y_pred_logits, axis=1)  # Get class indices with maximum logits

# Calculate accuracy and F1 score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

f1 = f1_score(y_test, y_pred, average='weighted')  # Use weighted average for imbalanced data
print(f"F1 Score (weighted): {f1:.4f}")


929/929 ━━━━━━━━━━━━━━━━━━━━ 1s 971us/step
Accuracy: 0.5633
F1 Score (weighted): 0.5330


## Trained on translated z embeddings



In [35]:
from sklearn.preprocessing import LabelEncoder
import scripts.classification as classification 
le = LabelEncoder()

X_train = source_df.iloc[:, 2:]
y_train = le.fit_transform(source_df.Website)
X_test = target_z
y_test = le.transform(target_df.Website)

# model = create_simple_dense_classifier(latent_dim, len(test_web_samples))
model.fit(X_train, y_train, batch_size = 128, epochs=10, shuffle=True)

InvalidArgumentError: Exception originated from

  File "/home/e19372/anaconda3/envs/doh_synth_env/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/home/e19372/anaconda3/envs/doh_synth_env/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
    self._run_once()
  File "/home/e19372/anaconda3/envs/doh_synth_env/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once
    handle._run()
  File "/home/e19372/anaconda3/envs/doh_synth_env/lib/python3.11/asyncio/events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
  File "/home/e19372/anaconda3/envs/doh_synth_env/lib/python3.11/site-packages/ipykernel/kernelbase.py", line 545, in dispatch_queue
    await self.process_one()
  File "/home/e19372/anaconda3/envs/doh_synth_env/lib/python3.11/site-packages/ipykernel/kernelbase.py", line 534, in process_one
    await dispatch(*args)
  File "/home/e19372/anaconda3/envs/doh_synth_en

In [33]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Convert logits to class labels
y_pred_logits = model.predict(X_test)
y_pred = np.argmax(y_pred_logits, axis=1)  # Get class indices with maximum logits

# Calculate accuracy and F1 score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

f1 = f1_score(y_test, y_pred, average='weighted')  # Use weighted average for imbalanced data
print(f"F1 Score (weighted): {f1:.4f}")

  1/929 ━━━━━━━━━━━━━━━━━━━━ 36s 40ms/step

929/929 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Accuracy: 0.2514
F1 Score (weighted): 0.2255


# Both translated and source z embeddings for training

In [23]:
from sklearn.preprocessing import LabelEncoder
import scripts.classification as classification 
le = LabelEncoder()


X_train = np.concatenate([source_z, get_z_translations(source_z, translator)])

y_train = le.fit_transform(source_df.Website)
y_train = np.concatenate([y_train, le.transform(source_df.Website)])

X_test = target_z
y_test = le.transform(target_df.Website)

model = create_simple_dense_classifier(latent_dim, len(test_web_samples))
model.fit(X_train, y_train, batch_size = 32, epochs=40, shuffle=True)

Epoch 1/40


/home/e19372/anaconda3/envs/doh_synth_env/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1857/1857 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.1131 - loss: 4.5472
Epoch 2/40
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.4210 - loss: 2.1812
Epoch 3/40
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.4987 - loss: 1.8455
Epoch 4/40
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5285 - loss: 1.7054
Epoch 5/40
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5466 - loss: 1.6341
Epoch 6/40
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.5520 - loss: 1.5916
Epoch 7/40
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5607 - loss: 1.5679
Epoch 8/40
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5715 - loss: 1.5280
Epoch 9/40
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5717 - loss: 1.5290
Epoch 10/40
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5772 - loss: 1.4975
Epoch 11/40
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5829 - loss: 1.4874
Epoch 12/40
1857/1857 ━━━━━━━━━━━━━━━━━━━

In [25]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Convert logits to class labels
y_pred_logits = model.predict(X_test)
y_pred = np.argmax(y_pred_logits, axis=1)  # Get class indices with maximum logits

# Calculate accuracy and F1 score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

f1 = f1_score(y_test, y_pred, average='weighted')  # Use weighted average for imbalanced data
print(f"F1 Score (weighted): {f1:.4f}")

929/929 ━━━━━━━━━━━━━━━━━━━━ 1s 875us/step
Accuracy: 0.5856
F1 Score (weighted): 0.5610
